In [1]:
%pwd  #look at the current work dir

'/home/raphael/MILA/ift6759/Solar-Irradiance-Prediction'

In [ ]:

#%cd   #change to the dir you want 
from src.data import dataloader as dl
from src.data import metadata as meta
metadata = meta.MetadataLoader(file_name="/home/raphael/MILA/ift6759/project1_data/catalog.helios.public.20100101-20160101.pkl")
reader = dl.ImageReader()
config = {}
config["LOCAL_PATH"] = "/home/raphael/MILA/ift6759/project1_data/hdf5v7_8bit/"
config["SKIP_MISSING"] = True
dataloader = dl.DataLoader(reader, config)
metaloader= metadata.load(meta.Station.BND,[36.62373, -116.01947, 1007])
dataset = dataloader.create_dataset(metaloader)
for sample in dataset:
    print (sample)
    break


/home/raphael/anaconda3/envs/ift6159-project1/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)
